In [ ]:
!python -m spacy download pt_core_news_sm

     |████████████████████████████████| 21.2MB 1.2MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp36-none-any.whl size=21186282 sha256=df6ee75af88f9b3333c2060ca53837f914e4409fe9f7db97038f1749b0b6cc23
  Stored in directory: /tmp/pip-ephem-wheel-cache-kenjx6eq/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [ ]:
import spacy
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors

w2v_modelo_cbow = KeyedVectors.load_word2vec_format("/content/drive/My Drive/curso_word2vec/modelo_cbow.txt")
w2v_modelo_sg = KeyedVectors.load_word2vec_format("/content/drive/My Drive/curso_word2vec/modelo_skipgram.txt")
artigo_treino = pd.read_csv("/content/drive/My Drive/curso_word2vec/treino.csv")
artigo_teste = pd.read_csv("/content/drive/My Drive/curso_word2vec/teste.csv")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"])

def tokenizador(texto):
    
    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower())

    
    return  tokens_validos

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
tokens = tokenizador(texto)
print(tokens)

['rio', 'janeiro', 'cidade', 'maravilhosa']


In [ ]:
def combinacao_de_vetores_por_soma(palavras, modelo):

    vetor_resultante = np.zeros((1,300))

    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)

        except KeyError:
            pass
                

    return vetor_resultante

vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
print(vetor_texto.shape)
print(vetor_texto)

(1, 300)
[[ 1.98463321e-01  6.81657493e-02  1.46854076e+00 -3.64254490e-01
   1.43774167e-01  8.67518850e-01  5.45219969e-01  1.10110116e+00
  -6.50840998e-01 -3.40783868e+00  1.01653042e+00  1.33011723e+00
   2.09624138e+00 -1.38510185e+00  1.47641352e+00  1.86899729e+00
  -1.59593132e+00  3.16831052e-01 -9.20437280e-01 -6.29614353e-01
   7.00041652e-02 -1.95808137e+00  7.94665724e-01 -7.24693198e-01
   1.39489579e+00  1.73469097e+00  1.26399602e+00 -1.10639864e+00
  -1.78198762e+00 -3.93549776e+00  2.81366953e+00 -2.87773656e-02
  -2.41185263e-01  2.14455798e-01  9.56606358e-01  1.02985160e+00
   1.33243561e+00  3.73910588e+00 -3.42612457e+00  2.09202743e+00
   2.08773389e+00  6.37439638e-01 -1.93942163e-01 -1.25078782e+00
  -1.34374684e+00  5.62602848e-01 -2.27634996e+00 -1.38811620e+00
   8.86206836e-01 -1.02591036e+00 -9.22529399e-01 -8.44487831e-01
   1.45723258e+00  6.67459078e-01  3.34969163e-01  9.08002794e-01
  -2.67864984e+00 -3.10524291e+00 -2.00997367e+00  3.95785213e-01
 

In [ ]:
def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))

    for i in range(x):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

    return matriz

matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo_cbow)
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape) 

(90000, 300)
(20513, 300)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(modelo, x_treino, y_treino, x_teste, y_teste):

    RL = LogisticRegression(max_iter = 800)
    RL.fit(x_treino, y_treino)
    categorias = RL.predict(x_teste)
    resultados = classification_report(y_teste, categorias)
    print(resultados)
    
    return RL

RL_cbow = classificador(w2v_modelo_cbow,
                        matriz_vetores_treino_cbow,
                        artigo_treino.category,
                        matriz_vetores_teste_cbow,
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.63      0.81      0.71      1698
     esporte       0.93      0.86      0.89      4663
   ilustrada       0.13      0.86      0.23       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.81      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [ ]:
matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)

RL_sg = classificador(w2v_modelo_sg,
                        matriz_vetores_treino_sg,
                        artigo_treino.category,
                        matriz_vetores_teste_sg,
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.81      0.72      0.76      6103
   cotidiano       0.64      0.81      0.71      1698
     esporte       0.93      0.88      0.90      4663
   ilustrada       0.14      0.89      0.24       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.75      0.84      0.79      2051

    accuracy                           0.80     20513
   macro avg       0.69      0.82      0.71     20513
weighted avg       0.82      0.80      0.81     20513



In [ ]:
import pickle

with open("/content/drive/My Drive/curso_word2vec/rl_cbow.pkl", "wb") as f:
    pickle.dump(RL_cbow, f)

In [ ]:
with open("/content/drive/My Drive/curso_word2vec/rl_sg.pkl", "wb") as f:
    pickle.dump(RL_sg, f)